# LiqPay API Notebook

## Setup

Load environment variables and code autoreload plugin.

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv -o .env

Imports

In [ ]:
from os import getenv
from uuid import uuid4
from pprint import pprint
from datetime import datetime, timedelta
from webbrowser import open_new_tab

from liqpy.client import Client
from liqpy.api.exceptions import LiqPayException
from liqpy.dev.testing import TestCard, gen_card_cvv, gen_card_expire
from liqpy.dev.server import LiqpayServer
from liqpy.constants import LIQPAY_TZ

Initialize client with public and private keys

In [ ]:
client = Client()
client

## Examples

### Forming a request to API

In [ ]:
order_id = uuid4()
order_id

Encode request and decode it as callback

In [ ]:
data, signature = client.encode(
    action="pay",
    amount=1,
    order_id=order_id,
    description="Test Encoding",
    currency="USD",
)

sep, end = "\n", "\n\n"
print("data:", data, sep=sep, end=end)
print("signature:", signature, sep=sep, end=end)

client.callback(data, signature)

### Make payments

In [ ]:
card_exp_month, card_exp_year = gen_card_expire(valid=True)
order_id = uuid4()

order_id

Pay request with test card

In [ ]:
client.pay(
    order_id=order_id,
    amount=1,
    currency="USD",
    description="Test Payment",
    card=TestCard.successful(),
    card_exp_month=card_exp_month,
    card_exp_year=card_exp_year,
    card_cvv=gen_card_cvv(),
    ip="127.0.0.1"
)

Refund the payment

In [ ]:
client.refund(order_id=order_id)

Get a payment status

In [ ]:
client.status(order_id)

add info to payment

In [ ]:
client.data(order_id, "Lorem Ipsum")

Failed payment

In [ ]:
try:
    card_exp_month, card_exp_year = gen_card_expire(valid=True)

    order_id = str(uuid4())
    client.pay(
        order_id=order_id,
        amount=1,
        currency="USD",
        description="Test Payment",
        card=TestCard.failure("9859"),
        card_exp_month=card_exp_month,
        card_exp_year=card_exp_year,
        card_cvv=gen_card_cvv(),
    )
except LiqPayException as e:
    print(e.code, e)
    print(e.response)
    pprint(e.details)
    raise e

### Webhooks

In [ ]:
action = "subscribe"
order_id = uuid4()

expire = timedelta(seconds=60)
timeout = (expire + timedelta(seconds=10)).total_seconds()

server_url = getenv("SERVER_URL") or None
server_url

Test checkout callback

In [ ]:
checkout_url = client.checkout(
    action,
    amount=1,
    order_id=order_id,
    description=f"test {action} checkout",
    currency="USD",
    expired_date=expire,
    subscribe_date_start=timedelta(days=7),
    subscribe_periodicity="month",
    # result_url="https://example.com/result",
    server_url=server_url,
)

print("checkout link\n", checkout_url)
open_new_tab(checkout_url)

if server_url is not None:
    with LiqpayServer(client=client, timeout=timeout) as server:
        pprint(server.handle_callback())

In [ ]:
client.status(order_id)

### Recurring payments

In [ ]:
order_id = uuid4()
order_id

Create subscription

In [ ]:
card_exp_month, card_exp_year = gen_card_expire(valid=True)

client.subscribe(
    amount=1,
    order_id=order_id,
    description="Test Subscribe",
    currency="USD",
    card=TestCard.successful(),
    card_exp_month=card_exp_month,
    card_exp_year=card_exp_year,
    card_cvv=gen_card_cvv(),
    # phone="+380661234567",
    subscribe_periodicity="month",
    subscribe_date_start=timedelta()
)

Cancel a recurring payment

In [ ]:
client.unsubscribe(order_id)

### Reports

In [ ]:
date_to = datetime.now(LIQPAY_TZ)
date_from = date_to - timedelta(hours=1)
print("from:", date_from)
print("to:", date_to)

Get payments report

In [ ]:
result = client.payments(date_from=date_from, date_to=date_to)
print("count:", len(result))
# pprint(result)

Reports in XML format

In [ ]:
from xml.dom.minidom import parseString as parse_xml

print(parse_xml(client.reports(date_from=date_from, date_to=date_to, format="xml")).toprettyxml())

Reports in CSV format

In [ ]:
data = client.reports(date_from=date_from, date_to=date_to, format="csv")
# print(data)

parse csv reports using standard module

In [ ]:
from csv import DictReader
from liqpy.models.report import Report

reports = [Report.from_dict(item) for item in DictReader(data.splitlines())]
pprint(reports[0])

parse csv reports using pandas

In [ ]:
# !pip install pandas
from io import StringIO
from liqpy.pandas import read_liqpay_csv

df = read_liqpay_csv(StringIO(data))
print(df.dtypes)
df